
# PyTorch e CIFAR-10

In [7]:
%cd ~/src/laboratori

/home/jovyan/src/laboratori


Librerie necessarie:
- `pytorch` 
    - https://pytorch.org/
    - https://docs.pytorch.org/docs/stable/index.html
- `numpy`
- `matplotlib`

In [8]:
import torch
import torchvision
import numpy as np

import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

#### Caricamento del dataset

Passaggi:
- definizione delle trasformazioni da applicare alle immagini in input
- selezione del dataset CIFAR-10
- selezione del batch size
- creazione dei generatori python (DataLoader) per l'uso del dataset
- definizione delle etichette di classe

In [10]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 4

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


#### Configurazione degli iperparametri

In [15]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 30
learning_rate = 0.1

#### Definizione della rete neurale

Creare un modulo pytorch (classe derivata da `nn.Module`) che definisce l'architettura della rete.  
Devono essere presenti due strati convoluzionali con pooling, concatenati infine a un MLP con 10 neuroni di output.

In [12]:
# Define the CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        # The input to the first fully connected layer will depend on the output size of the conv and pool layers
        # Assuming input 3x32x32:
        # After conv1 (kernel 5): 32-5+1 = 28. Output: 6x28x28
        # After pool1 (kernel 2, stride 2): 28/2 = 14. Output: 6x14x14
        # After conv2 (kernel 5): 14-5+1 = 10. Output: 16x10x10
        # After pool2 (kernel 2, stride 2): 10/2 = 5. Output: 16x5x5
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 16 channels * 5x5 feature map
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # 10 output classes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5) # Flatten the tensor for the fully connected layer
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x) # No activation here, CrossEntropyLoss will apply softmax
        return x

model = SimpleCNN().to(device)

#### Scelta della funzione Loss e dell'ottimizzatore

- Cross-entropy loss
- SGD optimizer

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

Implementare l'algoritmo di training della rete neurale sopra definita, ricordando che:
- per un certo numero di epoche, si itera su tutto il dataset
    - inoltre, i comandi `inputs.to(device), labels.to(device)` assicurano di utilizzare la GPU
- l'ottimizzatore va resettato a ogni iterazione, con `optimizer.zero_grad()`
- per ogni iterazione, si procede con il passo forward
- per ogni iterazione, si calcola la loss relativa a quell'output della rete
    - con `loss = criterion(outputs, labels)`
- per ogni iterazione, si propaga all'indietro la loss
- per ogni iterazione, si esegue uno step dell'ottimizzatore

In [14]:
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if (i + 1) % 2000 == 0: # Print every 2000 mini-batches
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 2000:.4f}')
            running_loss = 0.0
print('Finished Training')

Epoch [1/30], Step [2000/12500], Loss: 2.3042


KeyboardInterrupt: 

#### Inferenza

In [ ]:
# Visualization of some test images and predictions
def imshow(img):
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(test_loader)
images, labels = next(dataiter)

# Show images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]}' for j in range(batch_size)))

# Predict
images = images.to(device)
outputs = model(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]}' for j in range(batch_size)))

Altri esempi di addestramento reti con Pytorch:
https://docs.pytorch.org/examples/